In [1]:
import transformers
import torch

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
#model_id="mistral-community/Mixtral-8x22B-v0.1"

In [2]:
pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
    token='your_hugginface_token'
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
# context=''

# with open('test1.txt','r') as f:
#     for line in f:
#         context+=line[:-1]

In [4]:
import fitz

In [5]:
file='1-s2.0-S0966979513000241-main.pdf'
doc=fitz.open(file)

In [6]:
import re

toc=doc.get_toc()

pages=[]
for i,x in enumerate(toc):
    if(re.search('exper',x[1].lower())):
        pages.append(x[2])
        pages.append(toc[i+1][2])



In [7]:
context=''
for i in pages:
    page=doc[i-1]
    text=page.get_text('text')
    context+=text

In [8]:
messages = [
    {"role": "system", "content": context},
    {"role": "user", "content": "What is the synthesis temperature for Dy6Fe1.72In?"},
]

In [9]:
prompt = pipeline.tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
)

In [10]:
terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

In [11]:
outputs = pipeline(
    prompt,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.2,
    top_p=0.9,
)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [12]:
print(outputs[0]["generated_text"][len(prompt):])

According to the text, the synthesis temperature for Dy6Fe1.72In is 1170 K, and the annealing temperature is 870 K.


In [13]:
messages = [
    {"role": "system", "content": context},
    {"role": "user", "content": "Summery: what is the synthesis procedure?"},
]

In [14]:
prompt = pipeline.tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
)

In [15]:
terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

In [16]:
outputs = pipeline(
    prompt,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.2,
    top_p=0.9,
)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [17]:
print(outputs[0]["generated_text"][len(prompt):])

The synthesis procedure for the DyFeIn ternary alloys involves the following steps:

1. Preparation of starting materials: Ingots of dysprosium, iron, and indium with nominal purities better than 99.9% were used as starting materials.
2. Synthesis of alloys: The starting materials were melted in an arc-furnace on a water-cooled Cu-plate under an argon atmosphere (with sponge Ti as a getter) to form 30 alloys.
3. Homogenization: The buttons were turned over and re-melted two times to ensure homogeneity.
4. Weight loss measurement: The weight losses were smaller than 1% in all cases.
5. Annealing: The alloys were sealed in evacuated silica tubes and annealed at 870 K for 720 h and quenched in running water without breaking the tubes.
6. Single crystal growth: A special annealing sequence was used to grow single crystals of the Dy6Fe1.72In compound. The sample was heated to 1170 K for 5 h, annealed at this temperature for 4 h, cooled to 670 K at a rate of 10 K/hour, and then cooled to roo